In [1]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
file_name = "llama_3.2_val_results_20250402_210322.csv"

df = pd.read_csv(file_name)

In [3]:
def convert_target(value):
    if isinstance(value, str):
        match = re.search(r"tensor\((\d)\.", value)
        return int(match.group(1)) if match else None
    elif isinstance(value, (int, float)):
        return int(value)
    return None


df["target"] = df["target"].apply(convert_target)

In [4]:
def extract_answer_and_predict(response):
    matches = re.findall(r"<answer>(.*?)</answer>", response, re.IGNORECASE | re.DOTALL)
    answer_text = matches[-1].strip() if matches else ""
    prediction = 1 if "yes" in answer_text.lower() else 0
    return answer_text, prediction


df[["extracted_answer", "predicted_label"]] = df["response"].apply(
    lambda x: pd.Series(extract_answer_and_predict(x))
)

In [5]:
df

,id,target,response,timestamp,extracted_answer,predicted_label
0,1339.0,0,user\n\n\nYou are an expert in accident recons...,2025-04-02 18:25:52,No,0
1,168.0,1,user\n\n\nYou are an expert in accident recons...,2025-04-02 18:26:03,No,0
2,813.0,1,user\n\n\nYou are an expert in accident recons...,2025-04-02 18:26:38,No,0
3,1088.0,0,user\n\n\nYou are an expert in accident recons...,2025-04-02 18:27:12,No,0
4,1197.0,0,user\n\n\nYou are an expert in accident recons...,2025-04-02 18:27:45,No,0
...,...,...,...,...,...,...
295,1792.0,0,user\n\n\nYou are an expert in accident recons...,2025-04-02 21:01:23,No,0
296,171.0,1,user\n\n\nYou are an expert in accident recons...,2025-04-02 21:01:56,No,0
297,904.0,1,user\n\n\nYou are an expert in accident recons...,2025-04-02 21:02:28,No,0
298,667.0,1,user\n\n\nYou are an expert in accident recons...,2025-04-02 21:03:01,No,0


In [6]:
accuracy = accuracy_score(df["target"], df["predicted_label"])
accuracy

0.5466666666666666

In [7]:
conf_matrix = confusion_matrix(df["target"], df["predicted_label"])
class_report = classification_report(
    df["target"], df["predicted_label"], output_dict=True
)

conf_matrix, class_report

(array([[150,   0],
        [136,  14]]),
 {'0': {'precision': 0.5244755244755245,
   'recall': 1.0,
   'f1-score': 0.6880733944954128,
   'support': 150.0},
  '1': {'precision': 1.0,
   'recall': 0.09333333333333334,
   'f1-score': 0.17073170731707318,
   'support': 150.0},
  'accuracy': 0.5466666666666666,
  'macro avg': {'precision': 0.7622377622377623,
   'recall': 0.5466666666666666,
   'f1-score': 0.429402550906243,
   'support': 300.0},
  'weighted avg': {'precision': 0.7622377622377622,
   'recall': 0.5466666666666666,
   'f1-score': 0.4294025509062431,
   'support': 300.0}})